In [1]:
import pandas as pd
import helpers
import json
import numpy as np
import random
import math
import pickle
from sklearn.metrics.pairwise import linear_kernel
import collections

print('Loading interactions...')
interactions_train = helpers.load_interactions_unprocessed_df()
interactions_test = helpers.load_interactions_unprocessed_test_df()

print('Loading search map')
with open(f'./data/search_map.pickle', 'rb') as handle:
    search_map = pickle.load(handle)
print('Loading items')
items_dict = helpers.load_items()

Loading interactions...
Loading search map
Loading items


In [2]:
print('Starting processing...')

Starting processing...


In [23]:
%%time
import random
import dateutil.parser

items_counter = collections.defaultdict(int)
def process_search_rows(df):
    new_rows = collections.defaultdict(list)
    i = 0

    def process(row):
        nonlocal i
        i += 1
        if row['event_type'] != 'search' or type(row['item_id']) != str:
            return row
        str_id = helpers._normalize(row['item_id']).strip()
        items = search_map[str_id]
        
        item = items[items_counter[str_id]]
        items_counter[str_id] = (items_counter[str_id] + 1) % 5
        row['item_id'] = item
        
        if i % 100000 == 0:
            print(f"{i} processed")

        return row
    
    n_df = df
    for i in range(4):
        n_df = n_df.append(interactions_train[interactions_train['event_type'] == 'search'])
        print(f'Appended {i}')
    n_df = n_df.sort_values(by=['user_id', 'event_timestamp']).reset_index(drop=True)
    print(f'Sorted')
    n_df = n_df.apply(process, axis=1)
    return n_df

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 551 µs


In [ ]:
%%time
interactions_train = process_search_rows(interactions_train)
interactions_train

Appended 0
Appended 1
Appended 2
Appended 3
Sorted
100000 processed
200000 processed
300000 processed
400000 processed
500000 processed
600000 processed
700000 processed
800000 processed
900000 processed
1000000 processed
1100000 processed
1200000 processed
1300000 processed
1400000 processed
1500000 processed
1600000 processed
1800000 processed
1900000 processed
2000000 processed
2100000 processed
2200000 processed
2300000 processed
2500000 processed
2800000 processed
2900000 processed
3000000 processed
3100000 processed
3200000 processed
3300000 processed
3400000 processed
3500000 processed
3900000 processed
4000000 processed
4100000 processed
4200000 processed
4300000 processed
4400000 processed
4500000 processed
4600000 processed
4700000 processed
4800000 processed
4900000 processed
5000000 processed
5100000 processed
5200000 processed
5300000 processed
5400000 processed
5500000 processed
5600000 processed
5700000 processed
5900000 processed
6100000 processed
6300000 processed
6400

In [ ]:
complete_train = interactions_train.sort_values('user_id')
complete_train.to_csv('./data/interactions_train_complete2.csv', index=False)

In [5]:
%%time
interactions_test = process_search_rows(interactions_test)
interactions_test

600000 processed
700000 processed
800000 processed
900000 processed
1100000 processed
1200000 processed
1300000 processed
1400000 processed
1600000 processed
1900000 processed
2000000 processed
2300000 processed
2400000 processed
2700000 processed
3100000 processed
3400000 processed
3500000 processed
3600000 processed
3700000 processed
4000000 processed
4200000 processed
4300000 processed
5000000 processed
5100000 processed
CPU times: user 6min 19s, sys: 1min 15s, total: 7min 35s
Wall time: 7min 47s


,user_id,item_id,event_type,event_timestamp
0,0,1572239,view,2019-09-26T18:31:47.705-0400
1,0,1572239,view,2019-09-26T18:35:04.724-0400
2,0,1572239,view,2019-09-26T18:37:35.532-0400
3,0,1572239,view,2019-09-26T18:38:54.680-0400
4,0,1572239,view,2019-09-26T18:40:26.904-0400
...,...,...,...,...
5187749,177069,1657854,search,2019-10-10T14:40:10.487-0400
5187750,177069,888769,view,2019-10-10T14:40:21.691-0400
5187751,177069,419427,search,2019-10-10T14:40:25.402-0400
5187752,177069,765664,search,2019-10-10T19:38:15.871-0400


In [6]:
complete_test = interactions_test.sort_values('user_id')
complete_test.to_csv('./data/interactions_test_complete.csv', index=False)

In [7]:
a = helpers.load_interactions_df()

In [20]:
any(a.user_id.isna())

False